In [2]:
# Copyright (c) 2025, InfinityQ Technology Inc.

import numpy as np
import pandas as pd
import yfinance as yf
import logging
import itertools
from utils import *
from titanq import Model, Vtype, Target, S3Storage
import warnings
import json

import warnings

# Filter out FutureWarning messages due to yfinance using a deprecated pandas keyword
warnings.simplefilter(action='ignore', category=FutureWarning)

## Setting Credentials
The user should configure their TitanQ API key here. For very large problems, the user must also configure an AWS Access key, AWS Secret Access key and AWS Bucket Name.

We can also set the instance to test. You can see the available instances under the "instances" folder.

In [ ]:
logging.getLogger('botocore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)

# Enter your API key here
# Obtain your API key by contacting --> support@infinityq.tech
# Example: TITANQ_DEV_API_KEY = "00000000-0000-0000-0000-000000000000"
TITANQ_DEV_API_KEY = None

# Specify AWS keys and bucket name for solving very large problems
# AWS_ACCESS_KEY = "Access Key"
# AWS_SECRET_ACCESS_KEY = "Secret Access Key"
# AWS_BUCKET_NAME = "Bucket Name"

# Read in tickers and weights from chosen file
instance = "currencies_6"
currencies = read_instance(f"instances/{instance}")

# Number of nodes in graph
size = len(currencies)

## Setting Up The Problem
Here we pick an instance from the instances folder and read in the currency symbols that we are interested in. Then we draw forex (currency exchange) real-time data from Yahoo Finance, and we store the exchange rates from all the data provided in a dictionary. This may take a few minutes for our currencies_25 problem. 

We then load the exchange rate data into our dataframe, which we call our exchange rate matrix. Missing data is replaced with 0 to avoid invalid currency exchanges.

If you want to skip past loading live data for this, change the variable live_data to be False.

In [4]:
# Loading data from yfinance or using hardcoded data
live_data = False

# Initialize exchange rate dictionary
exchange_rates = {}

if live_data:
    # Generate all possible pairs 
    currency_pairs = [f"{b}=X" if a == "USD" else f"{a}{b}=X" for a,b in itertools.permutations(currencies, 2)]

    # Fetch data for all pairs (preventing logging for missing pairs in yfinance)
    logging.getLogger("yfinance").setLevel(logging.CRITICAL)
    forex_data = yf.Tickers(" ".join(currency_pairs))

    # Retrieve exchange rates for all pairs
    for pair in currency_pairs:
        ticker = forex_data.tickers.get(pair)
        if ticker:
            # Making names for exchange rates consistent
            name = pair
            if len(pair) < 8:
                name = "USD" + pair
            try:
                # Assigning rates to currency exchange names
                rate = ticker.history(period="1d")['Close'].iloc[-1]
                exchange_rates[name] = rate
            except Exception as e:
                # Filling missing rates with 0
                exchange_rates[name] = 0.

else: 
    # Loading exchange rates from file
    with open(f"instances/exch_rates/{instance}.json", "r") as f:
        exchange_rates = json.load(f)

# Converting rates into pandas dataframe
df = pd.DataFrame(index=currencies, columns=currencies)
for pair, rate in exchange_rates.items():
    base, quote = pair[:3], pair[3:6]
    df.at[base, quote] = rate

    # Replacing missing values with inverse exchange rate if it exists
    if rate == 0.0 and exchange_rates[quote + base + "=X"] != 0.0:
        df.at[base, quote] = 1.0 / exchange_rates[quote + base + "=X"]

# Filling NaN values with 0
df = df.where(pd.notna(df), 0.)

exch_rate_matrix = df.to_numpy().astype(np.float32)

## Create Bias Vector
To maximize arbitrage opportunity, we want to maximize the product of the exchange rates we use. We take the log of the exchange rates to convert the formulation into a summation. We also multiply each term of the objective function by -1 to pose it as a minimization problem for TitanQ. ```x``` is a vector of the length of the number of edges, and each entry is a binary variable encoding whether the edge will be used in our final solution.

We also want to prevent solutions that use non-existent edges, so we only keep track of edges that exist in the graph. This also helps preserve space and reduce the solve time, especially for graphs with a lot of nodes and not a lot of edges. Thus we shorten our exchange rate matrix and bias vector to only include non-zero values, and keep track of all of our edge names.

In [5]:
# List existing edges for TitanQ formulation
edges = [(i, j) for i in range(size) for j in range(size) if exch_rate_matrix[i,j] > 0]
num_edges = len(edges)

# Exchange rates for existing edges only
exch_rate_values = exch_rate_matrix[tuple(zip(*edges))]

# Bias vector for objective
bias = np.where(exch_rate_values > 0, -np.log(exch_rate_values), 1.).astype(np.float32)

edge_names = [currencies[i]+"/"+currencies[j] for i,j in edges]
print("Corresponding edges: ", edge_names)

Corresponding edges:  ['USD/EUR', 'USD/JPY', 'USD/GBP', 'USD/AUD', 'USD/CAD', 'EUR/USD', 'EUR/JPY', 'EUR/GBP', 'EUR/AUD', 'EUR/CAD', 'JPY/USD', 'JPY/EUR', 'JPY/GBP', 'JPY/AUD', 'JPY/CAD', 'GBP/USD', 'GBP/EUR', 'GBP/JPY', 'GBP/AUD', 'GBP/CAD', 'AUD/USD', 'AUD/EUR', 'AUD/JPY', 'AUD/GBP', 'AUD/CAD', 'CAD/USD', 'CAD/EUR', 'CAD/JPY', 'CAD/GBP', 'CAD/AUD']


## Constraint Matrices
Next, we formulate our constraint matrices and bounds.

The sum of all purchases of a currency is equal to the sum of all sales of a currency. This is called our flow constraint and is represented in our equality matrix.

We also forbid exchanging through a currency more than once. This is our inequality matrix. 

In [6]:
# Flow constraint matrix
eq_matrix = np.zeros((size, num_edges)).astype(np.float32)
for k, (i,j) in enumerate(edges):
    eq_matrix[i, k] = 1.
    eq_matrix[j, k] = -1.

# Removing rows of all zeros (only necessary when loading live data from yfinance to account for potentially missing exchange rates)
eq_matrix = eq_matrix[~np.all(eq_matrix == 0, axis = 1)]

eq_values = np.zeros(len(eq_matrix)).astype(np.float32)

# Number of exchanges constraint matrix
ineq_matrix = np.zeros((size, num_edges)).astype(np.float32)
for k, (i, j) in enumerate(edges):
    ineq_matrix[i, k] = 1

# Removing rows of all zeros (only necessary when loading live data from yfinance to account for potentially missing exchange rates)
ineq_matrix = ineq_matrix[~np.all(ineq_matrix == 0, axis = 1)]

ineq_bounds = np.full((len(ineq_matrix), 2), [0, 1]).astype(np.float32)

## Input Problem into TitanQ SDK

In [7]:
#############
# TitanQ Model
#############
model = Model(
    api_key=TITANQ_DEV_API_KEY
    # Insert storage_client parameter and specify corresponding AWS keys and bucket name for solving very large problems
    # storage_client=S3Storage(
    #     access_key=AWS_ACCESS_KEY,
    #     secret_key=AWS_SECRET_ACCESS_KEY,
    #     bucket_name=AWS_BUCKET_NAME
    # )
)

# Defining variables, objective, and constraints in TitanQ model
n = model.add_variable_vector('x', num_edges, Vtype.BINARY)
model.set_objective_matrices(None, bias, Target.MINIMIZE)

# Cycle constraints
model.add_equality_constraints_matrix(eq_matrix, eq_values) # Flow constraint
model.add_inequality_constraints_matrix(ineq_matrix, ineq_bounds) # Number of exchanges constraint

## Hyperparameters
The user should adjust the hyperparameters timeout_in_secs, Tmax, and penalty_scaling.

For this problem, penalty_scaling will impact TitanQ's solution quality. For small problems (where the size of ```x``` <= 30), penalty_scaling can be set to around 10. For the larger instance, adjust penalty_scaling to be around 45 as written. In general, penalty_scaling is proportional to the size of the problem.

In [8]:
# TitanQ Solver Hyperparameters
num_chains = 8
num_engines = 2
Tmin = 0.005

# For currencies_6, timeout ~2sec
# For currencies_25, timeout ~15sec
timeout_in_secs = 2

# For currencies_6, Tmax = ~122
# For currencies_25, Tmax = ~550
Tmax = 122
beta = (1/np.linspace(Tmin, Tmax, num_chains, dtype=np.float32)).tolist()

# For the currencies_6, penalty_scaling = ~10
# For the currencies_25, penalty_scaling = ~45
penalty_scaling = 10

# TitanQ solve
response = model.optimize(beta = beta, timeout_in_secs=timeout_in_secs, num_chains=num_chains, num_engines=num_engines, penalty_scaling=penalty_scaling)

## Outputting Results
The list of solutions for each engine can be accessed from the variable label defined before (in this case: ```x```). For each engine we print:
- The solution vector ```x```
- All profitable cycles
- The profit gained from each cycle
- The Ising energy of the solution (objective value)

Then we print the *best* valid cycle. This gives a series of currency exchanges for optimal arbitrage gain.

#### Example:
Best cycle ('HUX/USD', 'USD/RUB', 'RUB/HUX'): 0.017367341154281624

profit: 1.7367341154281624 %

energy: -0.004294343292713165

In [9]:
best_weight = 0
best_cycle = ()
# Keep track of the index and the weight of the best solution
best_idx = -1
print("-------- ALL ENGINE RESULTS --------")
for idx, solution in enumerate(response.x):
    print(f"\n--- Engine {idx + 1} ---")

    # Edges used in solution
    exchanges = [edge_names[i] for i, val in enumerate(solution) if val > 0.5]

    # If no edges used
    if not exchanges:
        print("No cycle found")
        continue

    # Calculate total profit from all cycles in solution
    cycles = find_exchange_cycles(exchanges)
    
    solution_profit, outputs = calculate_profit(cycles, currencies, exch_rate_matrix)

    # If no exchange cycle or arbitrage opportunity exists
    if all(value < 1 for value in outputs.values()):
        print("No profitable cycle found")
        continue
    
    # Print results if a profitable cycle is found
    for cycle, profit in outputs.items():
        if profit > 1:
            print("Cycle: ", cycle)
            print("Product of Exchange Rates: ", profit)
            print(f"Profit: {(profit - 1)* 100} %\n")
            if profit > best_weight:
                best_idx = idx
                best_weight = profit
                best_cycle = cycle

    # Ising energy        
    print("Energy:", response.computation_metrics().get('solutions_objective_value')[idx])

# Print the results of the best valid solution
print("\n-------- BEST VALID SOLUTION --------")
if best_idx == -1 or best_weight < 1:
    print("None of the engines returned profitable solutions!")
    print("Try adjusting the hyperparameters further to yield some valid solutions.")
else:
    solution = response.x[best_idx]
    print(f"--- Engine {best_idx + 1} ---")
    print("     x:", solution)
    print(f"Best Cycle: ", best_cycle)
    print("Product of Exchange Rates: ", best_weight)
    print("Best Profit: ", (best_weight - 1)*100, "%")
    print("Energy:", response.computation_metrics().get('solutions_objective_value')[best_idx]) 

-------- ALL ENGINE RESULTS --------

--- Engine 1 ---
Cycle:  ('AUD/USD', 'USD/GBP', 'GBP/EUR', 'EUR/AUD')
Product of Exchange Rates:  1.000033939007061
Profit: 0.003393900706094577 %

Energy: -3.395974636077881e-05

--- Engine 2 ---
Cycle:  ('AUD/USD', 'USD/GBP', 'GBP/AUD')
Product of Exchange Rates:  1.0000764531012354
Profit: 0.007645310123538529 %

Energy: -7.648766040802002e-05

-------- BEST VALID SOLUTION --------
--- Engine 2 ---
     x: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Best Cycle:  ('AUD/USD', 'USD/GBP', 'GBP/AUD')
Product of Exchange Rates:  1.0000764531012354
Best Profit:  0.007645310123538529 %
Energy: -7.648766040802002e-05
